In [1]:
import pandas as pd
import wrangle as wr
import summaries as s
from importlib import reload
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
%config InlineBackend.figure_format = 'retina'

pd.options.display.float_format = '{:,.2f}'.format

In [33]:
def modeling_preprocessing(df:pd.DataFrame, columns_to_keep=['customer_type', 'month_name','day_name', 'quarter'], regression:bool=True, np_array:bool=True):
    '''
    prepares data for the modeling
    Warning! Try not to change np_array default setting. Creating dummies for data frame is very risky! 
    The length of the train and test sets with dummies might be different
    
    Parameters:
        df: clean data frame
        regression: 
            True if regression modeling
            False if time series modeling
        np_array:
            True: return np.array of arrays with OneHotEncoding for X_train and X_test
            False: return data frames for X_train and X_test
    
    Returns:
        X_train, y_train, X_test, y_test
        
    '''
    df = prepare_df_preprocessing(df, columns_to_keep)
    train, test = wr.split_data(df)

    X_train, y_train, X_test, y_test = train.iloc[:, :-1], train.iloc[:,-1], test.iloc[:, :-1], test.iloc[:,-1]
    # change data types
    for col in X_train.columns:
        X_train[col] = pd.Categorical(X_train[col])
        X_test[col] = pd.Categorical(X_test[col])
    if regression:
        # regression time series
            if np_array:
                ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False).fit(X_train)
                X_train = ohe.transform(X_train)
                X_test = ohe.transform(X_test)
            else:
                X_train = create_dummies(X_train)
                X_test = create_dummies(X_test)
    else:
        # pure time series
        X_train = train.purchase_amount
        X_test = test.purchase_amount

    
    return X_train, y_train, X_test, y_test

In [3]:
def create_dummies(df):
    '''
    create dummy variables for all categorical columns
    this function might not work for modeling, as the test data set might not get all date features
    in this case use the parameter np_array = True in the preprocessing function
    
    Parameters:
        df -> train or test data set
    Return:
        df with encoded categorical varaibles
    '''
    dummies_q = pd.get_dummies(df.quarter, drop_first=True)
    dummies_m = pd.get_dummies(df.month, drop_first=True)
    dummies_w = pd.get_dummies(df.week, drop_first=True)
    dummies_d = pd.get_dummies(df.day_of_week, drop_first=True)
    df['is_school'] = np.where(df.customer_type == 'K-12', 1, 0)
    df['is_gov'] = np.where(df.customer_type == 'Local Goverment', 1, 0)
    df['is_edu'] = np.where(df.customer_type == 'Higher Ed', 1, 0)
    df['is_state'] = np.where(df.customer_type == 'State Agency', 1, 0)
    df = df[['is_school', 'is_gov', 'is_edu', 'is_state']]
    return pd.concat([df, dummies_d, dummies_m, dummies_w, dummies_q],axis=1)

In [25]:
def prepare_df_preprocessing(df:pd.DataFrame, columns_to_keep:list, target='purchase_amount') -> pd.DataFrame:
    '''
    Keep only needed columns
    Convert them to category type
    Parameters: 
        df: dataframe before splitting
    Return: 
        df: ready to split
    '''
    
    # change data types
    for col in columns_to_keep:
        df[col] = pd.Categorical(df[col])
        df[col] = pd.Categorical(df[col])
    columns_to_keep.append(target)
    df = df[columns_to_keep]
    return df

In [30]:
df = wr.get_clean_data(start2018=True)
sdf = s.get_summary_df(df)
sdf = wr.add_date_features(sdf)

In [ ]:
['customer_type', 'month_name','day_name', 'quarter', 'purchase_amount']

In [27]:
df = prepare_df_preprocessing(df, ['customer_type', 'month_name','day_name', 'quarter'])

In [38]:
X_train, y_train, X_test, y_test = modeling_preprocessing(df, regression=False)

In [40]:
X_train.shape

(235525, 2)

In [14]:
train, test = wr.split_data(df)
train = train[['customer_type', 'month_name','day_name', 'quarter', 'purchase_amount']]
test = test[['customer_type', 'month_name','day_name', 'quarter', 'purchase_amount']]
X_train, y_train, X_test, y_test = train.iloc[:, :-1], train.iloc[:,-1], test.iloc[:, :-1], test.iloc[:,-1]

In [16]:
for col in X_train.columns:
    X_train[col] = pd.Categorical(X_train[col])

In [41]:
# drafts to create functions

In [18]:
# !!!! Return numpy array, not a dataframe! But you can feed it into the model, no problem
ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False).fit(X_train)
X_train = ohe.transform(X_train).shape
X_test = ohe.transform(X_test).shape

In [23]:
ohe.transform(X_test).shape

(26361, 24)

In [22]:
X_train

,customer_type,month_name,day_name,quarter
order_date,,,,
2018-01-02,K-12,January,Tuesday,1
2018-01-02,K-12,January,Tuesday,1
2018-01-02,K-12,January,Tuesday,1
2018-01-02,K-12,January,Tuesday,1
2018-01-02,K-12,January,Tuesday,1
...,...,...,...,...
2021-12-30,K-12,December,Thursday,4
2021-12-30,Local Government,December,Thursday,4
2021-12-30,Local Government,December,Thursday,4


In [9]:
train

,customer_type,month_name,day_name,quarter,purchase_amount
order_date,,,,,
2018-01-02,K-12,January,Tuesday,1,"4,532.00"
2018-01-02,K-12,January,Tuesday,1,"2,860.00"
2018-01-02,K-12,January,Tuesday,1,187.04
2018-01-02,K-12,January,Tuesday,1,428.96
2018-01-02,K-12,January,Tuesday,1,63.36
...,...,...,...,...,...
2021-12-30,K-12,December,Thursday,4,"3,234.32"
2021-12-30,Local Government,December,Thursday,4,495.00
2021-12-30,Local Government,December,Thursday,4,660.00


In [264]:
train1, test1 = wr.split_data(sdf)
#X_train1, y_train1, X_test1, y_test1 = train1.iloc[:, :-1], train1.iloc[:,-1], test1.iloc[:, :-1], test1.iloc[:,-1]

In [261]:
train1.columns == test1.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [265]:
a, b, c, d = preprocessing(train1, test1)

In [269]:
list(a[0])

[0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [198]:
ohe1 = OneHotEncoder(drop='first', sparse=False).fit(X_train1)
lr1 = LinearRegression()
lr1.fit(ohe1.transform(X_train1), y_train1)
predictions1 = y_train1.to_frame()
predictions1['baseline'] = y_train1.mean()
mean_squared_error(y_train1, lr1.predict(ohe1.transform(X_train1))) ** 0.5,\
mean_squared_error(y_train1, predictions1.baseline) ** 0.5

(125105.03278176718, 125799.02545028755)

In [197]:
y_train1.mean()

31568.769158329145

In [193]:
ohe1.transform(X_train1)

<30285x76 sparse matrix of type '<class 'numpy.float64'>'
	with 129791 stored elements in Compressed Sparse Row format>

In [151]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

In [150]:
ohe = OneHotEncoder(drop='first', sparse=False).fit(X_train)
ohe.transform(X_train)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [137]:
print(ohe)

OneHotEncoder(drop='first')


In [ ]:
le = Lab

In [53]:
df

,customer_type,year,quarter,month,week,day_of_week,purchase_amount
order_date,,,,,,,
2018-01-02,K-12,2018,1,1,1,1,"4,532.00"
2018-01-02,K-12,2018,1,1,1,1,"2,860.00"
2018-01-02,K-12,2018,1,1,1,1,187.04
2018-01-02,K-12,2018,1,1,1,1,428.96
2018-01-02,K-12,2018,1,1,1,1,63.36
...,...,...,...,...,...,...,...
2022-11-25,Local Government,2022,4,11,47,4,"2,469.24"
2022-11-28,Local Government,2022,4,11,48,0,430.00
2022-11-28,Local Government,2022,4,11,48,0,525.78


In [49]:
ohe.transform(df).toarray().shape

(261886, 65351)

In [54]:
ohe.categories_

[array(['Higher Ed', 'K-12', 'Local Government', 'Other', 'State Agency'],
       dtype=object),
 array([2018, 2019, 2020, 2021, 2022]),
 array([1, 2, 3, 4]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53]),
 array([0, 1, 2, 3, 4, 5, 6]),
 array([-696740.88, -468750.  , -391124.56, ..., 2028000.  , 2406064.5 ,
        3723000.  ])]

In [69]:
y_test

order_date
2022-01-01      872.56
2022-01-01   32,728.16
2022-01-01   98,250.00
2022-01-01    8,152.38
2022-01-01    2,237.84
                ...   
2022-11-25    2,469.24
2022-11-28      430.00
2022-11-28      525.78
2022-11-29      418.96
2022-11-29    1,376.48
Name: purchase_amount, Length: 26361, dtype: float64

In [70]:
ohe = OneHotEncoder(drop='first').fit(X_train)

In [139]:
ohe.transform(X_train).toarray().shape

(235525, 79)

In [73]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(ohe.transform(X_train), y_train)

LinearRegression()

In [74]:
from sklearn.metrics import mean_squared_error

In [75]:
mean_squared_error?

In [78]:
mean_squared_error(y_train, lr.predict(ohe.transform(X_train))) ** 0.5

24027.65470925845

In [79]:
predictions = y_train.to_frame()
predictions['baseline'] = y_train.mean()

In [81]:
mean_squared_error(y_train, predictions.baseline) ** 0.5

24109.55733823772

In [93]:
train1 = train1.drop(['customer_name', 'customer_city', 'reseller_name', 'shipped_date', 'month_name', 'day_name', 'day_of_year'], axis=1)

#### Takeaway:
as expected, the model works much better on the summary data than on the original data. As our goal is prediction of daily sales, in makes more sence to use summary data frame that calculates sales for every company by the end of the day.

In [274]:
OneHotEncoder(sparse=False).fit_transform([['green'],['blue'],['red']])

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])